# Install

In [ ]:
import os
import shutil

In [ ]:
# check if `pytorch-CycleGAN-and-pix2pix` is already cloned
if not os.path.exists('pytorch-CycleGAN-and-pix2pix'):
	!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
os.chdir('./pytorch-CycleGAN-and-pix2pix')

In [ ]:
!pwd

# Datasets

Put the dataset in the `pytorch-CycleGAN-and-pix2pix/datasets` folder

(We have finished this part at the previous step when running `preprocess_dataset.ipynb`)

Each dataset should have the following directory structure:

```
datasets
└── testing_dataset
	├── testA
```

# Testing (Baseline)
convert the testA images to domainB images

use the trained model to inference the testA images

## Load testing data folder

select the folder where the testing data is located.

In [ ]:
test_dir = './datasets/public_testing_dataset_1'
# the directory may only contain `testA` subfolder

## Check the weights
After training, you should have a folder with the weights of the model. 

It should be located in the `pytorch-CycleGAN-and-pix2pix/checkpoints` folder.

For example, in our previous training, we have the following weights:
`checkpoints/ROAD_RIVER_pix2pix/latest_net_G.pth`

In [ ]:
!ls checkpoints/

In [ ]:
print(not os.path.exists('./checkpoints/ROAD_RIVER_pix2pix'))

In [ ]:
!pwd

In [ ]:
# if there is no pre-trained model, use our pre-trained model
if not os.path.exists('./checkpoints/ROAD_RIVER_pix2pix'):
    !bash ../scripts/download_pretrained_weight.sh


## Inference with the pre-trained model with single mode

- `--dataroot`: the folder where the testing data is located
- `--name`: the name of the model
- `--model`: the model mode
- `--netG`: the weights of the generator
- `--direction`: the direction of the model
- `--dataset_mode`: single (which we don't need to prepare the paired data)
- `--num_test`: the number of testing data (default is 50)

In [ ]:
! python test.py --dataroot ./datasets/public_testing_dataset_1/testA/ --name ROAD_RIVER_pix2pix --model test --netG unet_256 --direction AtoB --dataset_mode single --norm batch --num_test 1000

## Transform the results into AI CUP format

The results are stored in `./ROAD_RIVER_pix2pix/test_latest/images/`.

And there are 2 types of results:
- `{Prefix}_real` (domainA)
- `{Prefix}_fake` (domainB)

Store the `{Prefix}_fake.png` as `{Prefix}.jpg` to `./ROAD_pix2pix/test_latest/submission/`.

In [ ]:
# store the fake images to the `./results/ROAD_RIVER_pix2pix/test_latest/submission/` folder
import os
import cv2
import shutil

source_folder = './results/ROAD_RIVER_pix2pix/test_latest/images'
target_folder = './results/ROAD_RIVER_pix2pix/test_latest/submission'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

for image_name in os.listdir(source_folder):
    if 'fake' in image_name:
        new_name = image_name.replace('_fake.png', '.jpg')
        shutil.copy(os.path.join(source_folder, image_name), os.path.join(target_folder, new_name))

In [ ]:
# resize the image as 420x240
import os
import cv2

for image_name in os.listdir(target_folder):
    img = cv2.imread(os.path.join(target_folder, image_name))
    img = cv2.resize(img, (420, 240))
    cv2.imwrite(os.path.join(target_folder, image_name), img)
print("Finished resizing images")
print(f"Size: {len(os.listdir(target_folder))}")

In [ ]:
# zip the fake images
shutil.make_archive(target_folder, 'zip', target_folder)

# Enhanced Method (2 domain-specific models)

## Load The Test Data

In [ ]:
import zipfile

dataset_zip = 'dataset/public_testing_dataset_1.zip'
dataset_dir = 'dataset/public_testing_dataset_1'

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

In [ ]:
!pwd

## Preprocess The Image to match the model input

The label_img folder contains images `.png`, and all the images does not contains the ground truth, so we need to add the fake paired images, and concatenate to the original images to create the paired images.

In [ ]:
import os

# source path
dataset_label_img_dir = 'dataset/public_testing_dataset_1/label_img'
# destination path
# 2 folder: 1 for RIVER, 1 for ROAD
public_test_road_dir = 'dataset/public_test_ROAD/test'
public_test_river_dir = 'dataset/public_test_RIVER/test'

# create destination directory if not exist
if not os.path.exists(public_test_river_dir):
	os.makedirs(public_test_river_dir)
if not os.path.exists(public_test_road_dir):
	os.makedirs(public_test_road_dir)

In [ ]:
# process each image in the source directory
# and copy to destination directory
import cv2
import numpy as np

for filename in os.listdir(dataset_label_img_dir):
	# read the image
	img = cv2.imread(os.path.join(dataset_label_img_dir, filename))

	# concatenate the image with itself horizontally
	img_double = np.concatenate((img, img), axis=1)

	# determine the destination directory based on the filename
	if 'RI' in filename:
		destination_dir = public_test_river_dir
	elif 'RO' in filename:
		destination_dir = public_test_road_dir

	# write the new image to the destination directory
	cv2.imwrite(os.path.join(destination_dir, filename), img_double)

print('Done!')

## Copy the test folder to the model test input folder

In [ ]:
import os
import shutil

# Source directory
source_river_dir = './dataset/public_test_RIVER/test'
source_road_dir = './dataset/public_test_ROAD/test'
# Target directory
target_dir = './AI cup demo code/pytorch-CycleGAN-and-pix2pix/datasets/'

# copy the source folder to the target directory
shutil.copytree(source_river_dir, os.path.join(target_dir, 'public_test_RIVER/test'))
shutil.copytree(source_road_dir, os.path.join(target_dir, 'public_test_ROAD/test'))

## Test The Model

In [ ]:
# change directory to the pix2pix code
import os
os.chdir('./pytorch-CycleGAN-and-pix2pix')

In [ ]:
# test the 2 models
! python test.py --dataroot ./datasets/public_test_ROAD --name ROAD --model pix2pix --direction AtoB --num_test 1000
! python test.py --dataroot ./datasets/public_test_RIVER --name RIVER --model pix2pix --direction AtoB --num_test 1000

In [ ]:
print(f"Size: {len(os.listdir('./results/ROAD/test_latest/images/'))}")

## Transform the results into AI CUP format

The results are stored in `./ROAD_pix2pix/test_latest/images/`.

And there are 2 types of results: `real_A` and `fake_B`.

The `real_A` is the original image, and the `fake_B` is the transformed image.

Store the `{Prefix}_fake_B.png` as `{Prefix}.jpg` to `./ROAD_pix2pix/test_latest/submission/`.

In [ ]:
# store the fake images to the `./results/ROAD_RIVER_pix2pix/test_latest/submission/` folder
import os
import cv2
import shutil

source_folder = './results/ROAD/test_latest/images'
target_folder = './results/ROAD/test_latest/submission'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

for image_name in os.listdir(source_folder):
    if 'fake' in image_name:
        new_name = image_name.replace('_fake_B.png', '.jpg')
        shutil.copy(os.path.join(source_folder, image_name), os.path.join(target_folder, new_name))

In [ ]:
# resize the image as 420x240
import os
import cv2

for image_name in os.listdir(target_folder):
    img = cv2.imread(os.path.join(target_folder, image_name))
    img = cv2.resize(img, (420, 240))
    cv2.imwrite(os.path.join(target_folder, image_name), img)
print(f"Size: {len(os.listdir(target_folder))}")

In [ ]:
# zip the fake images
shutil.make_archive(target_folder, 'zip', target_folder)

## Transform the RIVER

In [ ]:
# store the fake images to the `./results/ROAD_RIVER_pix2pix/test_latest/submission/` folder
import os
import cv2
import shutil

source_folder = './results/RIVER/test_latest/images'
target_folder = './results/RIVER/test_latest/submission'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

for image_name in os.listdir(source_folder):
    if 'fake' in image_name:
        new_name = image_name.replace('_fake_B.png', '.jpg')
        shutil.copy(os.path.join(source_folder, image_name), os.path.join(target_folder, new_name))

In [ ]:
# resize the image as 420x240
import os
import cv2

for image_name in os.listdir(target_folder):
    img = cv2.imread(os.path.join(target_folder, image_name))
    img = cv2.resize(img, (420, 240))
    cv2.imwrite(os.path.join(target_folder, image_name), img)
print(f"Size: {len(os.listdir(target_folder))}")

## Combine the ROAD and RIVER submission

In [ ]:
source_road_dir = './results/ROAD/test_latest/submission'
source_river_dir = './results/RIVER/test_latest/submission'

target_dir = './results/submission'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

for image_name in os.listdir(source_road_dir):
	shutil.copy(os.path.join(source_road_dir, image_name), os.path.join(target_dir, image_name))
for image_name in os.listdir(source_river_dir):
	shutil.copy(os.path.join(source_river_dir, image_name), os.path.join(target_dir, image_name))

In [ ]:
# zip the fake images
shutil.make_archive(target_dir, 'zip', target_dir)